# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.
Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте accuracy на тестовой выборке самостоятельно.

Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

сalls — количество звонков,

minutes — суммарная длительность звонков в минутах,

messages — количество sms-сообщений,

mb_used — израсходованный интернет-трафик в Мб,

is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

План решения:

1)Откроем и изучим файл

2)Разобьем данные на выборки

3)сследуем модели

4)проверим модель на тестовой выборке

## Откроем и изучим файл

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
import warnings

In [4]:
warnings.filterwarnings("ignore")

In [72]:
df = pd.read_csv('/datasets/users_behavior.csv')
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


## Разобьем данные на выборки

In [6]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

In [7]:
features.shape

(3214, 4)

In [8]:
target.shape

(3214,)

In [9]:
# разбиваем данные на тренировочные и данные для теста и валидации

features_train, features_test_valid, target_train, target_test_valid = train_test_split(
    features, target, test_size=0.4, random_state=12345) 

In [10]:
# разбиваем данные на тестовые и валидационные

features_valid, features_test,target_valid, target_test = train_test_split(
    features_test_valid, target_test_valid, test_size=0.5, random_state=12345) 

Мы разбили данные на тренировочные, валидационные и тестовые в пропорциях 3:1:1

In [11]:
features_train.shape

(1928, 4)

In [12]:
features_valid.shape

(643, 4)

In [13]:
features_test.shape

(643, 4)

In [14]:
target_train.shape

(1928,)

In [15]:
target_valid.shape

(643,)

In [16]:
target_test.shape

(643,)

## Исследуем модели

Исследуем 3 модели:

Дерево решений

Случайный лес

Логистическую регрессию

In [17]:
# проверим долю правильных ответов для дерева решений на валидационной выборке

best_model = None
best_result = 0
max_depth=0
for depth in range(1, 6):
	model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
	model.fit(features_train,target_train) 
	predictions = model.predict(features_valid) 
	result = accuracy_score(target_valid, predictions) 
	if result > best_result:
		best_model_DecisionTree = model
		best_result = result
        
        
print("Accuracy лучшей модели:", best_result)

Accuracy лучшей модели: 0.7853810264385692


Как мы видим девево решений дает нам долю правильных ответов в 0.78

In [18]:
# проверим долю правильных ответов для случайного леса на валидационной выборке

best_model = None
best_result = 0
for depth in range(1, 6):
    for est in range(10, 51,10):
        model = RandomForestClassifier(random_state=12345, max_depth=depth,n_estimators=est)
        model.fit(features_train,target_train) 
        predictions = model.predict(features_valid) 
        result = accuracy_score(target_valid, predictions) 
        if result > best_result:
            best_model_RandomForest = model
            best_result = result
        
print("Accuracy лучшей модели:", best_result)

Accuracy лучшей модели: 0.7947122861586314


Как мы видим модель случайного леса дает нам долю правильных ответов в 0.79, что больше модели дерева решений.

In [19]:
# проверим долю правильных ответов для логистической регрессии на валидационной выборке

model = LogisticRegression(random_state=12345) 
model.fit(features_train,target_train) 
predictions = model.predict(features_valid) 
result = accuracy_score(target_valid, predictions) 

print("Accuracy модели:", result)

Accuracy модели: 0.7589424572317263


Как мы видим модель логистической регрессии дает нам долю правильных ответов в 0.758, это самый низкий уровень из трех моделей.

Выводы: лучшая модель это случайный лес

## Проверим модель на тестовой выборке

In [20]:
# проверим долю правильных ответов для модели случайного леса на тестовой выборке

predictions = best_model_RandomForest.predict(features_test)
result = accuracy_score(target_test, predictions)

print("Accuracy лучшей модели на тестовой выборке:", result)

Accuracy лучшей модели на тестовой выборке: 0.7838258164852255


Мы получили долю правильных ответов на тестовой выборке равную 0.78, что меньше чем на валидационной, но всего на 0.01

## (бонус) Проверьте модели на адекватность

In [69]:
df['is_ultra'].value_counts(normalize=True)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

In [74]:
# точность правильной константной модели должна быть равна доли бОльшего класса.

model = DummyClassifier(strategy="constant",constant=0) 
model.fit(features_train,target_train) 
predictions = model.predict(features_test) 
accuracy = accuracy_score(target_test, predictions) 

print("Accuracy константной модели на тестовой выборке:", accuracy)

Accuracy константной модели на тестовой выборке: 0.6842923794712286


Мы провели тест константной модели подставляя в прогноз только класс 0 (которого больше всего в столбце). В результате получили чдолю правильных ответов 0.68, что больше простого угадывания 50/50, но меньше 0.78

Как мы видим при савнении нашей модели с константной наша модешь выигрывает. 

## Вывод

 Мы построили модель для задачи классификации, которая выберает подходящий тариф.
 
 В результате тестирования мы получили долю правильных ответов на тестовой выборке равную 0.78, что больше 0.75


## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняеxся без ошибок
- [x] Ячейки с кодом раxположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гиперрррррпараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
